## Data Types

In [1]:
#quirks
#dividing 2 ints returns int (version 2 only)
#spaces are important in python
#variables are references
#flexibility (tradeoff btwn rules and conventions with flexibility)

In [2]:
#ask ppl if they know how to get their computers shell/terminal open, loops, how to write functions, if they are familiar with the basics of python

In [3]:
#contingency for when most ppl know python - go thru example of building a model in python, training it, understanding overfitting and how to evaluate a model
#how to encode information with dummy variables and why (dist btwn 1and 2 != dist btwn red, green)

In [4]:
#give my computer a 5, it looks at it and thinks yup, i see thats an integer with value 5, and gives it back

In [5]:
#integer
5

5

In [6]:
#i give my computer a small math problem, and it gives back the answers

In [7]:
#float
8.3 + 5

13.3

In [8]:
#explain that python is trying to keep the result the same type.

In [9]:
#example of python quirk (version 2 only)
10/3

3

In [10]:
#string
'string'

'string'

In [11]:
string

NameError: name 'string' is not defined

In [12]:
'string' + 'another word'

'stringanother word'

In [13]:
'string' + 5

TypeError: cannot concatenate 'str' and 'int' objects

In [14]:
#boolean
5 == 5

True

In [15]:
'string' == 'another word'

False

In [16]:
#list

In [17]:
[1, 2, 3]

[1, 2, 3]

In [18]:
['one', 'two', 3]

['one', 'two', 3]

In [19]:
[4, 5 'six']

SyntaxError: invalid syntax (<ipython-input-19-c0cc6467b91d>, line 1)

## Functions

In [20]:
#talk about the importance of spaces in python
def test(word):
    print 'Hello'
    print word
print 'Whatever'

Whatever


In [21]:
test('World')

Hello
World


## Variables -  store stuff for later

In [22]:
#notice that my computer isn't giving the input back to me right away. its storing it
variable1 = [4, 5, 6]

In [23]:
variable1[1]

5

In [24]:
variable2 = variable1

In [25]:
variable2

[4, 5, 6]

In [26]:
variable1[1] = 10

In [27]:
variable1

[4, 10, 6]

In [28]:
variable2

[4, 10, 6]

In [29]:
variable3 = [test, 2]

In [30]:
variable3[0]

<function __main__.test>

In [31]:
variable3[0]()

TypeError: test() takes exactly 1 argument (0 given)

## Loops

In [32]:
count = 0
for i in 'how now brown cow':
    if i == 'w':
        count = count + 1
    print count


0
0
1
1
1
1
2
2
2
2
2
3
3
3
3
3
4


## Classes

In [33]:
5 + 5

10

In [34]:
5 + True

6

In [35]:
5 + 'word'

TypeError: unsupported operand type(s) for +: 'int' and 'str'

In [36]:
class testClass():
    def __init__(self, number):
        self.color = 'red'
        self.value = number

    def __add__(self, string):
        return str(self.value) + string

In [37]:
test5 = testClass(2)

In [38]:
testClass(2) + 'word'

'2word'

## Data Science Models

In [45]:
from sklearn.datasets import base
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
from data_prep import parse_tickets, parse_payouts
from sklearn.cross_validation import train_test_split
from sklearn.metrics import confusion_matrix

In [40]:
data = pd.read_json('data.json')

In [41]:
model_data = data[['acct_type', 'ticket_types', 'previous_payouts', 'user_created', 'user_age', 'event_start', 'event_created', 'approx_payout_date', 'name_length']]

model_data.loc[:,'avg_ticket_sold_cnt'] = model_data.loc[:,'ticket_types'].map(lambda x: parse_tickets(x)[3])
model_data.loc[:,'event_age_at_start(days)'] = (model_data.loc[:,'event_start'] - model_data['event_created'])/86400.0
model_data.loc[:,'total_payout_hist'] = model_data.loc[:,'previous_payouts'].map(lambda x: parse_payouts(x)[2])
model_data.loc[:,'payout'] = model_data.loc[:,'ticket_types'].map(lambda x: parse_tickets(x)[4])
model_data.loc[:,'num_payouts_hist'] = model_data.loc[:,'previous_payouts'].map(lambda x: parse_payouts(x)[0])
model_data.loc[:,'avg_ticket_cost'] = model_data.loc[:,'ticket_types'].map(lambda x: parse_tickets(x)[1])

model_data = model_data.drop(['ticket_types', 'previous_payouts', 'event_start', 'event_created'], axis=1)

/home/michael/anaconda2/lib/python2.7/site-packages/numpy/core/fromnumeric.py:2909: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/michael/anaconda2/lib/python2.7/site-packages/numpy/core/_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/home/michael/anaconda2/lib/python2.7/site-packages/pandas/core/indexing.py:337: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/home/michael/anaconda2/lib/python2.7/site-packages/pandas/core/indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydat

In [42]:
label_map = {'fraudster': 1, 'fraudster_event': 1, 'locked': 1, 'premium': 1, 'spammer_limited': 1, 'spammer_noinvite': 1, 'spammer_warn': 1, \
             'tos_warn': 1, 'premium': 0}

model_data.loc[:,'label'] = model_data['acct_type'].map(label_map)

In [43]:
model_data = model_data.dropna()
y = model_data['label'].values
X = model_data.drop(['label', 'acct_type'], axis=1)
X_train, X_test, y_train, y_test = train_test_split(X.as_matrix(), y)

In [46]:
model = RandomForestClassifier()
model.fit(X_train, y_train)
confusion_matrix(y_test, model.predict(X_test))

array([[3089,   23],
       [ 166,  264]])

In [47]:
def my_cost(estimator, X, y):
    compare = pd.DataFrame(data=[y, estimator.predict(X), -X[:, 8]], index=['actual', 'pred', 'cost'])
    compare = compare.T
    compare['confusion'] = zip(compare['actual'], compare['pred'] == compare['actual'])
    compare['count'] = 1
    conf_map = {(0.0, False): 'FP', (0.0, True): 'TN', (1.0, False): 'FN', (1.0, True): 'TP'}
    compare['confusion'] = compare['confusion'].map(conf_map)
    compare.loc[compare['confusion'] == 'TP', 'cost'] = -500
    compare.loc[compare['confusion'] == 'FP', 'cost'] = -500
    compare.loc[compare['confusion'] == 'TN', 'cost'] = 0
    total_cost = compare.cost.agg('sum')
    conf_matrix = compare[['confusion', 'count']].groupby('confusion').agg('count')
    return total_cost, conf_matrix

In [48]:
my_cost(model, X_test, y_test)

(-150464.0,            count
 confusion       
 FN           166
 FP            23
 TN          3089
 TP           264)